# Exercise 1

### Necessary imports

In [1]:
import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus import semcor
from nltk.corpus.reader.wordnet import Synset
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
import math
import re
import random
from scipy import stats as st
nltk.download('wordnet')
nltk.download('semcor')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\lores\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package semcor to
[nltk_data]     C:\Users\lores\AppData\Roaming\nltk_data...
[nltk_data]   Package semcor is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lores\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\lores\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lores\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## First Part
- The exercise consists of implementing three similarity measures based on WordNet:
    - WU & Palmer similarity
    - Shortest Path similarity
    - Leacock & Chodorow similarity
- For each of these similarity measures, calculate Spearman's correlation indices and Pearson's correlation indices between the results obtained and the 'target' results in the dataset.

### Dataset loading

In [2]:
# Reading the file
df = pd.read_csv('dataset\wordsim353.csv')
display(df)

,Word 1,Word 2,Human (mean)
0,love,sex,6.77
1,tiger,cat,7.35
2,tiger,tiger,10.00
3,book,paper,7.46
4,computer,keyboard,7.62
...,...,...,...
348,shower,flood,6.03
349,weather,forecast,8.34
350,disaster,area,6.25
351,governor,office,6.34


### Support functions

Defining the function to get the max depth of the synset

In [3]:
def max_depth_synset(synset: Synset) -> int:
    return max(len(path) for path in synset.hypernym_paths())

#### Defining the function to get the least common subsumer of two synsets

We loop through the paths of the first synset, and for each path we check if it is in the second synset. If it is, we return it.
If the synsets have no common subsumer, we return the max depth.

In [4]:
def LCS(syn1: Synset, syn2: Synset) -> Synset | None:
    path1 = syn1.hypernym_paths()
    path2 = syn2.hypernym_paths()
    result = []

    # We loop through the paths of the first synset
    for pat1 in path1:
        x = pat1
        x.reverse()
        found = False
        for syn in x:
            if(not found):
                # We loop through the paths of the second synset
                for pat2 in path2:
                    if(syn in pat2):
                        result.append(syn)
                        found = True
                        break
            else:
                break
    if not result:
            return None
    else:
         max_depth = max(max_depth_synset(s) for s in result)
         max_hypernym = max(s for s in result if max_depth_synset(s) == max_depth)
         return max_hypernym

#### Path Length

We get the least common ancestor of the two synsets, and then we calculate the path length of each synset to the least common subsumer. We sum the two min path lengths and return the result.

In [5]:
def path_len(syn1: Synset, syn2: Synset) -> int:
    path1 = syn1.hypernym_paths()
    path2 = syn2.hypernym_paths()
    result_syn1 = []
    result_syn2 = []

    common_ancestor = LCS(syn1,syn2)

    if(common_ancestor is None):
        return None
    else:
        for pat1 in path1:
            if(common_ancestor in pat1):
                result_syn1.append(pat1.index(syn1)-pat1.index(common_ancestor))
        for pat2 in path2:
            if(common_ancestor in pat2):
                result_syn2.append(pat2.index(syn2)-pat2.index(common_ancestor))
        return min(result_syn1) + min(result_syn2)
  

In [6]:
depth_max = max(max(len(hyp_path) for hyp_path in ss.hypernym_paths()) for ss in wn.all_synsets())

### Wu Palmer

$$ LCS(s1, s2) = {2 * depth(LCS(s1,s2)) \over depth(s1) + depth(s2)} $$

In [7]:
def wu_palmer(syn1: Synset, syn2: Synset) -> float:
    # We get all the synsets of the two words
    depht1 = max_depth_synset(syn1)
    depht2 = max_depth_synset(syn2)
    
    # Finding the least common subsumer of the two synsets
    common_ancestor = LCS(syn1,syn2)
    if common_ancestor is None:
        return 0
    return 2 * max_depth_synset(common_ancestor) / ( depht1 + depht2 )

### Shortest Path

$$ LCS(s1, s2) = 2 * depthMax - len(s1,s2) $$

In [8]:
def shortest_path(syn1: Synset, syn2: Synset) -> int:
    pat = path_len(syn1,syn2)
    if pat is None:
        return 0
    elif (pat == 0):
        return 2 * depth_max
    elif (pat == depth_max):
        return 0
    else:
        return 2 * depth_max - pat

### Leakcock & Chodorow

$$ sim_LC(s1, s2) = -log(\frac{len(s1,s2)}{2*depthMax}) $$

In [9]:
def leakcock_chodorow (syn1: Synset, syn2: Synset) -> float:
    pat = path_len(syn1,syn2)
    if pat is None:
        return 0
    elif (pat == 0):
        return -math.log((1)/(2 * depth_max + 1))
    else:
        return -math.log((pat)/(2 * depth_max))

### Cycle for every synset of the words

In [10]:
def synsets_cycle(word1: str, word2: str, metric: callable) -> float | str:
    syn1 = wn.synsets(word1)
    syn2 = wn.synsets(word2)
    result = []
    for s1 in syn1:
        for s2 in syn2:
            result.append(metric(s1,s2))
    if not result:
        return 0
    else:
        return max(result)

In [11]:
# Test
print("WU Palmer between dog and cat:", synsets_cycle('dog','cat', wu_palmer))
print("WU Palmer between stock and live:", synsets_cycle('stock','live', wu_palmer))
print("Shortest Path between Jerusalem and Palestinian:", synsets_cycle('Jerusalem','Palestinian', shortest_path))
print("Leakcock Chodorow between dog and cat:", synsets_cycle('dog','cat', leakcock_chodorow))

WU Palmer between dog and cat: 0.8571428571428571
WU Palmer between stock and live: 0.2857142857142857
Shortest Path between Jerusalem and Palestinian: 24
Leakcock Chodorow between dog and cat: 2.3025850929940455


### Loop over all pair of words in the dataset and i print the three similarity measures

In [12]:
# Loop through the dataset and calculate the similarity for each pair of words using the three metrics and i add the result to a new dataframe
df_metrics = pd.DataFrame(columns=['Word 1', 'Word 2', "Human (mean)", "WU Palmer", "Shortest Path", "Leakcock Chodorow"])
for index, row in df.iterrows():
    df_metrics.loc[len(df_metrics)] = {'Word 1': row['Word 1'], 'Word 2': row['Word 2'], "Human (mean)": row['Human (mean)'], "WU Palmer": synsets_cycle(row['Word 1'], row['Word 2'], wu_palmer) * 10, "Shortest Path": (synsets_cycle(row['Word 1'], row['Word 2'], shortest_path) /40) * 10, "Leakcock Chodorow": (synsets_cycle(row['Word 1'], row['Word 2'], leakcock_chodorow) / math.log(41)) * 10}
display(df_metrics)

,Word 1,Word 2,Human (mean),WU Palmer,Shortest Path,Leakcock Chodorow
0,love,sex,6.77,9.230769,9.75,9.933507
1,tiger,cat,7.35,9.655172,9.75,9.933507
2,tiger,tiger,10.00,10.000000,10.00,10.000000
3,book,paper,7.46,8.750000,9.50,8.066983
4,computer,keyboard,7.62,8.235294,9.25,6.975136
...,...,...,...,...,...,...
348,shower,flood,6.03,6.363636,9.00,6.200459
349,weather,forecast,8.34,1.333333,6.75,3.026547
350,disaster,area,6.25,5.000000,8.00,4.333935
351,governor,office,6.34,5.263158,7.75,4.016766


### Calculation of correlation with Spearman and Pearson metrics between the human similarity and the similarity calculated by the three metrics

In [13]:
# Calculate the correlation between the human similarity and the similarity calculated by the three metrics

print("-------------------------------------------")
print("WU Palmer Metric:")
spearman_correlation = st.spearmanr(df_metrics["WU Palmer"], df_metrics["Human (mean)"])
print("Spearman Correlation:", spearman_correlation)
if spearman_correlation[1] < 0.05:
    print("The distribution of the WU Palmer metric is significantly different from the distribution of the Human Similarity.\n")
else:
    print("The distribution of the WU Palmer metric is almost the same as the distribution of the Human Similarity.\n")
pearson_correlation = st.pearsonr(df_metrics["WU Palmer"], df_metrics["Human (mean)"])
print("Pearson Correlation:", pearson_correlation)
if pearson_correlation[1] < 0.05:
    print("The distribution of the WU Palmer metric is significantly different from the distribution of the Human Similarity.")
else:
    print("The distribution of the WU Palmer metric is almost the same as the distribution of the Human Similarity.")
print("-------------------------------------------")

print("Shortest Path Metric:")
spearman_correlation = st.spearmanr(df_metrics["Shortest Path"], df_metrics["Human (mean)"])
print("Spearman Correlation:", spearman_correlation)
if spearman_correlation[1] < 0.05:
    print("The distribution of the Shortest Path metric is significantly different from the distribution of the Human Similarity.\n")
else:
    print("The distribution of the Shortest Path metric is almost the same as the distribution of the Human Similarity.\n")
pearson_correlation = st.pearsonr(df_metrics["Shortest Path"], df_metrics["Human (mean)"])
print("Pearson Correlation:", pearson_correlation)
if pearson_correlation[1] < 0.05:
    print("The distribution of the Shortest Path metric is significantly different from the distribution of the Human Similarity.")
else:
    print("The distribution of the Shortest Path metric is almost the same as the distribution of the Human Similarity.")
print("-------------------------------------------")

print("Leakcock Chodorow Metric:")
spearman_correlation = st.spearmanr(df_metrics["Leakcock Chodorow"], df_metrics["Human (mean)"])
print("Spearman Correlation:", spearman_correlation)
if spearman_correlation[1] < 0.05:
    print("The distribution of the Leakcock Chodorow metric is significantly different from the distribution of the Human Similarity.\n")
else:
    print("The distribution of the Leakcock Chodorow metric is almost the same as the distribution of the Human Similarity.\n")
pearson_correlation = st.pearsonr(df_metrics["Leakcock Chodorow"], df_metrics["Human (mean)"])
print("Pearson Correlation:", pearson_correlation)
if pearson_correlation[1] < 0.05:
    print("The distribution of the Leakcock Chodorow metric is significantly different from the distribution of the Human Similarity.")
else:
    print("The distribution of the Leakcock Chodorow metric is almost the same as the distribution of the Human Similarity.")
print("-------------------------------------------")

-------------------------------------------
WU Palmer Metric:
Spearman Correlation: SignificanceResult(statistic=0.34531446776235897, pvalue=2.5376440594705738e-11)
The distribution of the WU Palmer metric is significantly different from the distribution of the Human Similarity.

Pearson Correlation: PearsonRResult(statistic=0.28732609798215053, pvalue=3.895408936985221e-08)
The distribution of the WU Palmer metric is significantly different from the distribution of the Human Similarity.
-------------------------------------------
Shortest Path Metric:
Spearman Correlation: SignificanceResult(statistic=0.2883010732370922, pvalue=3.4876313366885126e-08)
The distribution of the Shortest Path metric is significantly different from the distribution of the Human Similarity.

Pearson Correlation: PearsonRResult(statistic=0.16411938908972765, pvalue=0.0019775111177550756)
The distribution of the Shortest Path metric is significantly different from the distribution of the Human Similarity.
---

## Second Part

- Implementing Lesk's algorithm (not from NLTK).
    1. Extract 50 sentences from the SemCor corpus (corpus annotated with WN synsets) and disambiguate (at least) one noun per sentence. Calculate the accuracy of the implemented system based on the senses annotated in SemCor.
    SemCor is available at the URL: http://web.eecs.umich.edu/~mihalcea/downloads.html
    2. Randomize the selection of the 50 sentences and the selection of the term to be disambiguated, and return the average accuracy over (for example) 10 runs of the program.

### Defining support function for cleaning the sentences

In [14]:
# I clean the sentences removing the punctuation and the stop words

def clean_sentence(sentence: str | list) -> list:
    sentence_without_puntuaction = re.sub(r'[^\w\s]', '', str(sentence)).strip()
    word_tokens = word_tokenize(sentence_without_puntuaction)
    filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
    return filtered_sentence

### Defining the Lesk Algorithm

We take all the synsets of the word.
Then make a for loop for each synset computing the overlap between the gloss and the context(sentence).
We return the synset with the highest overlap.

In [15]:
# I use the Lesk algorithm to find the best sense of the word in the sentence

def lesk(word: str, sentence: list, clean: bool) -> Synset:
    synsets = wn.synsets(word)
    if not synsets:
        return None
    best_sense = synsets[0]
    max_overlap = 0
    for synset in synsets:
        overlap = 0
        # Cleaning example in wordnet 
        for definition_word in clean_sentence(synset.definition()) if clean else synset.definition().split():
            if definition_word in sentence:
                overlap += 1
        for example_word in clean_sentence(synset.examples()) if clean else synset.examples():
            if clean:
                if example_word in sentence:
                    overlap += 1
            else:
                for example_word in example_word.split():
                    if example_word in sentence:
                        overlap += 1
        if overlap > max_overlap:
            max_overlap = overlap
            best_sense = synset
        
    return best_sense

### Defining the cycle for disambiguate the sostantive words in the sentences. 
I use the Lesk Algorithm for the best synset and I compare it with the synset of the SemCor corpus. 

To verify accuracy, we used two different methods. 

- With the first (**get_accuracy_pos_tag**) one we take the sentence, calculate the pos tagging with the nltk libraries and go to calculate the accuracy with lesk.
- With the second (**get_accuracy_from_corpus**) we go to take the nouns from the annotated corpus of semCor and then calculate the accuracy with lesk

In [16]:
# I select the sostantive words with pos_tag and I use the lesk algorithm to find the best synset
# I use the synset of the semcor corpus to compare the results

def get_accuracy_pos_tag(index_list: list, sentence_list: list, sentence_tagged_list: list, clean: bool) -> float:
    points = 0
    total = 0
    for index in index_list:
        # We get the sentence from the dataframe
        sentence = sentence_list.iloc[index]["sentences"]

        # We get the tagged sentence from the dataframe
        list_of_sentence_tagged = sentence_tagged_list.iloc[index]["tagged_sentences"]

        # We get the pos_tag of the sentence
        sentence_tagged = nltk.pos_tag(clean_sentence(sentence)) if clean else nltk.pos_tag(word_tokenize(' '.join(sentence)))
        for word, tag in sentence_tagged:
            # Only for noun
            if tag == 'NN':
                # We get the best synset
                best_synset = lesk(word, clean_sentence(sentence) if clean else sentence, clean)

                # If there is no synset we skip the word
                if best_synset is None:
                    break

                # We compare the synset with the synset of the semcor corpus
                for tag in list_of_sentence_tagged:
                    if tag.pos()[0][0] == word and tag.pos()[0][1] == 'NN' and hasattr(tag.label(), 'synset') and best_synset == tag.label().synset():
                        # If the synset are the same we add a point
                        points += 1 
                
                # We add a total, because we have to calculate the accuracy
                total += 1 
    print("Total number of words to disambiguate:", total)
    print("Number of words correctly disambiguated:", points)
    accuracy = points/total
    print("Accuracy:", accuracy)
    return accuracy


# I select the sostantive words directly from the semcor corpus and I use the lesk algorithm to to find the best synset.
# I use the synset of the semcor corpus to compare the results

def get_accuracy_from_corpus(index_list: list, sentence_list: list, sentence_tagged_list: list, clean: bool) -> float:
    points = 0
    total = 0
    for index in index_list:
        sentence = sentence_list.iloc[index]["sentences"]
        list_of_sentence_tagged = sentence_tagged_list.iloc[index]["tagged_sentences"]
        for tag in list_of_sentence_tagged: 
            if hasattr(tag, 'label') and hasattr(tag.label(), 'synset'): 
                # We get the synset of the semcor corpus
                true_synset_word = tag.label().synset()

                # Only for noun
                if true_synset_word.name().split('.')[1] == 'n': 
                    # We get the lemma of the word
                    word_lemma = tag.label().name()
                    best_synset = lesk(word_lemma, clean_sentence(sentence) if clean else sentence, clean)
                    if best_synset is None:
                        break 
                    if best_synset == true_synset_word:
                        points += 1 
                    total += 1
    print("Total number of words to disambiguate:", total)
    print("Number of words correctly disambiguated:", points)
    accuracy = points/total
    print("Accuracy:", accuracy)
    return accuracy

### Defining the range of index for extracting the sentences and the two pandas dataframe of sentences and tagged sentences

In [17]:
# I extract only first 50 sentences
list_of_index = range(0,50)

In [18]:
# For efficiency, I create two pandas dataframe to store the sentences and the tagged sentences
# Accessing directly to the corpus is very slow
sentences_semcor = pd.DataFrame({"sentences": semcor.sents()})
tagged_sentences_semcor = pd.DataFrame({'tagged_sentences': list(semcor.tagged_sents(tag='both'))})

### Extracting the first 50 sentences without randomization

In [19]:
print("-------------------------------------------")
print("Sostantive words extracted from the corpus\n")
get_accuracy_from_corpus(list_of_index, sentences_semcor, tagged_sentences_semcor, True)
print("-------------------------------------------")
print("Sostantive words extracted from the POS Tagging\n")
get_accuracy_pos_tag(list_of_index, sentences_semcor, tagged_sentences_semcor, True)
print("-------------------------------------------")
print("-------------------------------------------")
print("Sostantive words extracted from the corpus without cleaning the sentences\n")
get_accuracy_from_corpus(list_of_index, sentences_semcor, tagged_sentences_semcor, False)
print("-------------------------------------------")
print("Sostantive words extracted from the POS Tagging without cleaning the sentences\n")
get_accuracy_pos_tag(list_of_index, sentences_semcor, tagged_sentences_semcor, False)
print("-------------------------------------------")

-------------------------------------------
Sostantive words extracted from the corpus

Total number of words to disambiguate: 321
Number of words correctly disambiguated: 209
Accuracy: 0.6510903426791277
-------------------------------------------
Sostantive words extracted from the POS Tagging

Total number of words to disambiguate: 193
Number of words correctly disambiguated: 85
Accuracy: 0.44041450777202074
-------------------------------------------
-------------------------------------------
Sostantive words extracted from the corpus without cleaning the sentences

Total number of words to disambiguate: 321
Number of words correctly disambiguated: 149
Accuracy: 0.46417445482866043
-------------------------------------------
Sostantive words extracted from the POS Tagging without cleaning the sentences

Total number of words to disambiguate: 197
Number of words correctly disambiguated: 78
Accuracy: 0.39593908629441626
-------------------------------------------


### I run the algorithm 50 times and I calculate the average accuracy with randomized extraction of sentences

In [20]:
accuracy_list_corpus = []
accuracy_list_pos_tag = []
accuracy_list_corpus_unclean = []
accuracy_list_pos_tag_unclean = []

for i in range(50):
    # I select 50 random sentences in the range from 0 to the total number of tagged sentences
    random_index = random.sample(range(0, tagged_sentences_semcor.size), 50)
    print("-------------------------------------------")
    print("Iteraction number:", i + 1)
    print("-------------------------------------------")
    print("Sostantive words extracted from the corpus\n")
    accuracy_list_corpus.append(get_accuracy_from_corpus(random_index, sentences_semcor, tagged_sentences_semcor, True))
    print("-------------------------------------------")
    print("Sostantive words extracted from the POS Tagging\n")
    accuracy_list_pos_tag.append(get_accuracy_pos_tag(random_index, sentences_semcor, tagged_sentences_semcor, True))
    print("-------------------------------------------")
    print("Sostantive words extracted from the corpus without cleaning the sentences\n")
    accuracy_list_corpus_unclean.append(get_accuracy_from_corpus(random_index, sentences_semcor, tagged_sentences_semcor, False))
    print("-------------------------------------------")
    print("Sostantive words extracted from the POS Tagging without cleaning the sentences\n")
    accuracy_list_pos_tag_unclean.append(get_accuracy_pos_tag(random_index, sentences_semcor, tagged_sentences_semcor, False))

print("-------------------------------------------")
print("Mean accuracy from the corpus:", np.mean(accuracy_list_corpus))
print("Mean accuracy from the POS Tagging:", np.mean(accuracy_list_pos_tag))
print("Mean accuracy from the corpus without cleaning the sentences:", np.mean(accuracy_list_corpus_unclean))
print("Mean accuracy from the POS Tagging without cleaning the sentences:", np.mean(accuracy_list_pos_tag_unclean))

-------------------------------------------
Iteraction number: 1
-------------------------------------------
Sostantive words extracted from the corpus

Total number of words to disambiguate: 90
Number of words correctly disambiguated: 52
Accuracy: 0.5777777777777777
-------------------------------------------
Sostantive words extracted from the POS Tagging

Total number of words to disambiguate: 139
Number of words correctly disambiguated: 15
Accuracy: 0.1079136690647482
-------------------------------------------
Sostantive words extracted from the corpus without cleaning the sentences

Total number of words to disambiguate: 90
Number of words correctly disambiguated: 31
Accuracy: 0.34444444444444444
-------------------------------------------
Sostantive words extracted from the POS Tagging without cleaning the sentences

Total number of words to disambiguate: 153
Number of words correctly disambiguated: 13
Accuracy: 0.08496732026143791
-------------------------------------------
Ite